In [1]:
!pip install -r requirements.txt

   ---------------------------------------- 0.0/18.0 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/18.0 MB 12.2 MB/s eta 0:00:02
   --------- ------------------------------ 4.5/18.0 MB 11.2 MB/s eta 0:00:02
   ------------- -------------------------- 6.3/18.0 MB 11.7 MB/s eta 0:00:02
   ---------------- ----------------------- 7.6/18.0 MB 9.4 MB/s eta 0:00:02
   ---------------------- ----------------- 10.2/18.0 MB 9.8 MB/s eta 0:00:01
   --------------------------- ------------ 12.6/18.0 MB 10.2 MB/s eta 0:00:01
   --------------------------------- ------ 14.9/18.0 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------  17.6/18.0 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 18.0/18.0 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.29.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.29.3 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. OR-Tools, test.

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

df = pd.read_csv('./data/results_100yrs_25.01.24.csv')

# Fixed start location (longitude, latitude)
s_loc = np.array([127.077356, 37.312437])

# Example loop to process each time and cluster
for t in df['time'].unique():
    print(f"Processing Time: {t}")
    df_time = df[df['time'] == t]

    for cluster in df_time['cluster'].unique():
        df_cluster = df_time[df_time['cluster'] == cluster]
        print(f"Processing Cluster: {cluster} ({len(df_cluster)})")

        if len(df_cluster) < 1:
            print("Skipping: Not enough points to route.", '\n')
            continue

        # Extract coordinates and prepend start location
        locations = np.vstack([s_loc, df_cluster[['longitude', 'latitude']].values])
        num_locations = len(locations)

        # Compute Euclidean distance matrix
        distance_matrix = squareform(pdist(locations, metric='euclidean'))
        distance_matrix = np.round(distance_matrix * 1000).astype(int)  # Scale up for precision

        # Create OR-Tools model
        manager = pywrapcp.RoutingIndexManager(num_locations, 1, 0)  # Start at fixed start location
        routing = pywrapcp.RoutingModel(manager)

        # Distance callback function
        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return distance_matrix[from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Set search parameters
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH
        search_parameters.time_limit.seconds = 60  # Maximize search time
        search_parameters.solution_limit = 500  # Force OR-Tools to explore more options
        search_parameters.log_search = True  # Enable debugging logs

        # Solve
        solution = routing.SolveWithParameters(search_parameters)

        if solution:
            # Extract route
            index = routing.Start(0)
            route = []
            while not routing.IsEnd(index):
                route.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))  # Append depot

            # Compute total travel distance
            total_distance = sum(distance_matrix[route[i], route[i+1]] for i in range(len(route)-1))

            print(f"Optimal Route for Cluster {cluster}: {' → '.join(map(str, route))}")
            print(f"Total Travel Distance: {total_distance / 1000} km\n")
        else:
            print("No solution found.\n")


Processing Time: 07:30
Processing Cluster: 1 (3)
Optimal Route for Cluster 1: 0 → 1 → 3 → 2 → 0
Total Travel Distance: 0.929 km

Processing Cluster: 2 (9)
Optimal Route for Cluster 2: 0 → 6 → 1 → 2 → 5 → 9 → 3 → 4 → 8 → 7 → 0
Total Travel Distance: 0.698 km

Processing Cluster: 3 (1)
Optimal Route for Cluster 3: 0 → 1 → 0
Total Travel Distance: 0.764 km

Processing Cluster: 4 (5)
Optimal Route for Cluster 4: 0 → 3 → 4 → 2 → 1 → 5 → 0
Total Travel Distance: 0.552 km

Processing Cluster: 5 (4)
Optimal Route for Cluster 5: 0 → 2 → 4 → 1 → 3 → 0
Total Travel Distance: 0.515 km

Processing Cluster: 6 (3)
Optimal Route for Cluster 6: 0 → 3 → 1 → 2 → 0
Total Travel Distance: 0.484 km

Processing Cluster: 7 (4)
Optimal Route for Cluster 7: 0 → 2 → 3 → 4 → 1 → 0
Total Travel Distance: 0.622 km

Processing Cluster: 8 (4)
Optimal Route for Cluster 8: 0 → 1 → 4 → 2 → 3 → 0
Total Travel Distance: 0.199 km

Processing Cluster: 9 (4)
Optimal Route for Cluster 9: 0 → 4 → 3 → 1 → 2 → 0
Total Travel Dis